# CUDA for DUM DUMS

openai/triton seems to be the new hotness for efficiently running jupyter kernels

I tried reading the paper and barely understood a single word: prefecting, SRAM, linearization, blocks, warps, scheduling blcoks on warps and torps on warps

In [2]:
import torch
import triton
import triton.language as tl

def add(x: torch.Tensor, y: torch.Tensor):
    # We need to preallocate the output
    output = torch.empty_like(x)
    n_elements = output.numel()
    
    # grid denotes the number of kernels that can run in parallel
    # it's a function of the total number of the length of the vector / block size
    # block size is the number of concurrent elements we can handle at once, block because it works in multiple dimensions
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']),)
    
    # x, y, output are actually pointers to the tensors, think of it as copying by reference
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)
    # We return a handle to output but, since `torch.cuda.synchronize()` hasn't been called, the kernel is still
    # running asynchronously at this point.
    return output

In [3]:
from rich import inspect

In [7]:
type(add)

function

This is easy enough and if we run this function through a debubgger we'll know everything we need to know. `numel()` is just a function to get the number of elements. We need an output that's of the same size as the input because that's what vector addition is

Where things get confusing is the `add_kernel[]()` part, so it's a dictionary? That I'm indexing with a `grid` and that `grid` is a `lambda` function that divides the `n_elements` by a block size?

So let's step through this in a debugger by running something. Whoops turns out we can't actually trigger a breakpoint inside of `add_kernel` so what the heck is going on? What is a program? What's a block start? Offset? Masks. So confused :O


In [8]:
@triton.jit
def add_kernel(
    x_ptr,  # *Pointer* to first input vector
    y_ptr,  # *Pointer* to second input vector
    output_ptr,  # *Pointer* to output vector
    n_elements,  # Size of the vector
    BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process
                 # NOTE: `constexpr` so it can be used as a shape value
):
    # There are multiple 'program's processing different data. We identify which program
    # we are here
    pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0
    # This program will process inputs that are offset from the initial data.
    # for instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # Create a mask to guard memory operations against out-of-bounds accesses
    mask = offsets < n_elements
    # Load x and y from DRAM, masking out any extra elements in case the input is not a
    # multiple of the block size
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    output = x + y
    # Write x + y back to DRAM
    tl.store(output_ptr + offsets, output, mask=mask)

In [9]:
type(add_kernel)

triton.code_gen.JITFunction

In [11]:
inspect(add_kernel, methods=True)

╭──────────────────────────── JITFunction(__main__:add_kernel) ─────────────────────────────╮
│ def (*args, **kwargs):                                                                    │
│                                                                                           │
│       annotations = {4: <class 'triton.language.core.constexpr'>}                         │
│      arg_defaults = [                                                                     │
│                         <class 'inspect._empty'>,                                         │
│                         <class 'inspect._empty'>,                                         │
│                         <class 'inspect._empty'>,                                         │
│                         <class 'inspect._empty'>,                                         │
│                         <class 'inspect._empty'>                                          │
│                     ]                                                                     │
│         arg_names = ['x_ptr', 'y_ptr', 'output_ptr', 'n_elements', 'BLOCK_SIZE']          │
│         bin_cache = {}                                                                    │
│        cache_hook = None                                                                  │
│         cache_key = 'a13fe84bedaab4be158c7841ee3daf1b2-.-0-.-0--01e1cfad2157e0272927976a… │
│        constexprs = [4]                                                                   │
│ do_not_specialize = []                                                                    │
│              hash = 'a13fe84bedaab4be158c7841ee3daf1b2-.-0-.-0--01e1cfad2157e0272927976a… │
│            inline = True                                                                  │
│            kernel = None                                                                  │
│ kernel_decorators = []                                                                    │
│            module = '__main__'                                                            │
│               src = "def add_kernel(\n    x_ptr,  # *Pointer* to first input vector\n     │
│                     y_ptr,  # *Pointer* to second input vector\n    output_ptr,  #        │
│                     *Pointer* to output vector\n    n_elements,  # Size of the vector\n   │
│                     BLOCK_SIZE: tl.constexpr,  # Number of elements each program should   │
│                     process\n                 # NOTE: `constexpr` so it can be used as a  │
│                     shape value\n):\n    # There are multiple 'program's processing       │
│                     different data. We identify which program\n    # we are here\n    pid │
│                     = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0\n     │
│                     # This program will process inputs that are offset from the initial   │
│                     data.\n    # for instance, if you had a vector of length 256 and      │
│                     block_size of 64, the programs\n    # would each access the elements  │
│                     [0:64, 64:128, 128:192, 192:256].\n    # Note that offsets is a list  │
│                     of pointers\n    block_start = pid * BLOCK_SIZE\n    offsets =        │
│                     block_start + tl.arange(0, BLOCK_SIZE)\n    # Create a mask to guard  │
│                     memory operations against out-of-bounds accesses\n    mask = offsets  │
│                     < n_elements\n    # Load x and y from DRAM, masking out any extra     │
│                     elements in case the input is not a\n    # multiple of the block      │
│                     size\n    x = tl.load(x_ptr + offsets, mask=mask)\n    y =            │
│                     tl.load(y_ptr + offsets, mask=mask)\n    output = x + y\n    # Write  │
│                     x + y back to DRAM\n    tl.store(output_ptr + offsets, output,        │
│                     mask=mask)\n"                 

Ok so what's  JITFunction?

It seems to have a few interesting properties

Some defaut `arg_names` refering to the inputs `x_ptr, y_ptr`, the `output_ptr`, the number of elements `n_elements` and the number of elements each kernel will operate over concurrently i.e `BLOCK_SIZE`

The function can be parsed

In [12]:
add_kernel.parse()

In [13]:
import ast

print(ast.dump(add_kernel.parse(), indent=4))

Module(
    body=[
        FunctionDef(
            name='add_kernel',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x_ptr'),
                    arg(arg='y_ptr'),
                    arg(arg='output_ptr'),
                    arg(arg='n_elements'),
                    arg(
                        arg='BLOCK_SIZE',
                        annotation=Attribute(
                            value=Name(id='tl', ctx=Load()),
                            attr='constexpr',
                            ctx=Load()))],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='pid', ctx=Store())],
                    value=Call(
                        func=Attribute(
                            value=Name(id='tl', ctx=Load()),
                            attr='program_id',
                     

The AST we just printed is likely then optimized by some core engine but we'll get back to this in a second

In [15]:
add_kernel.fn(5,5,5,5,1)

ValueError: Did you forget to add @triton.jit ? (`_builder` argument must be provided outside of JIT functions.)

In [16]:
# We still would like to understand what a program ID is so let's try just try

pid = tl.program_id(axis=0)

ValueError: Did you forget to add @triton.jit ? (`_builder` argument must be provided outside of JIT functions.)

Well that's kinda sad, we're not going to able to run this program it feels like so let's just try to understand at least what's the type of `pid` by looking at the source code

Just remember that `_builder` in the error, we'll get back to it

In [38]:
from inspect import getsource
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(getsource(tl.program_id))

('@builtin\n'
 'def program_id(axis, _builder=None):\n'
 '    """\n'
 '    Returns the id of the current program instance along the given '
 ':code:`axis`.\n'
 '\n'
 '    :param axis: The axis of the 3D launch grid. Has to be either 0, 1 or '
 '2.\n'
 '    :type axis: int\n'
 '    """\n'
 '    # if axis == -1:\n'
 '    #     pid0 = program_id(0, _builder)\n'
 '    #     pid1 = program_id(1, _builder)\n'
 '    #     pid2 = program_id(2, _builder)\n'
 '    #     npg0 = num_programs(0, _builder)\n'
 '    #     npg1 = num_programs(0, _builder)\n'
 '    #     return pid0 + pid1*npg0 + pid2*npg0*npg1\n'
 '    axis = _constexpr_to_value(axis)\n'
 '    return semantic.program_id(axis, _builder)\n')


Ok we already learnt a few things
1. `axis` needs to be one of `0, 1, 2` as it refers to the axes of a 3D grid. Makes sense if we're working with tensors
2. `program_id` will return a `semantic.program_id` and can optionally take in a `_builder parameter`

In our case `axis = 1` so let's go down to that last line of code and figure out what a `semantic.program_id` is

In [41]:
pp.pprint(getsource(tl.semantic.program_id))

('def program_id(axis: int, builder: ir.builder) -> tl.tensor:\n'
 '    return tl.tensor(builder.create_get_program_id(axis), tl.int32)\n')


Interesting..

So the `pid` is actually a `tl.tensor` but not a `torch.Tensor` with a dtype of `int32`

We've also now finally hit some `C` code, the `ir.builder` is imported from

We have two paths to go down now, what's a `tl.tensor` and what's an `ir`


In [43]:
from triton._C.libtriton.triton import ir

In [45]:
inspect(ir, methods=True)

╭───── <module 'triton._C.libtriton.triton.ir'> ──────╮
│              CA = CACHE_MODIFIER.CA                 │
│              CG = CACHE_MODIFIER.CG                 │
│     EVICT_FIRST = EVICTION_POLICY.EVICT_FIRST       │
│      EVICT_LAST = EVICTION_POLICY.EVICT_LAST        │
│            NONE = CACHE_MODIFIER.NONE               │
│          NORMAL = EVICTION_POLICY.NORMAL            │
│        argument = def argument(...)                 │
│       ATOMIC_OP = def ATOMIC_OP(...) Members:       │
│       attribute = def attribute(...)                │
│  attribute_kind = def attribute_kind(...) Members:  │
│     basic_block = def basic_block(...)              │
│     bb_iterator = def bb_iterator(...)              │
│      block_type = def block_type(...)               │
│         builder = def builder(...)                  │
│  CACHE_MODIFIER = def CACHE_MODIFIER(...) Members:  │
│        constant = def constant(...)                 │
│  constant_float = def constant_float(...)           │
│    constant_int = def constant_int(...)             │
│         context = def context(...)                  │
│ EVICTION_POLICY = def EVICTION_POLICY(...) Members: │
│        function = def function(...)                 │
│   function_type = def function_type(...)            │
│     instruction = def instruction(...)              │
│    integer_type = def integer_type(...)             │
│          module = def module(...)                   │
│        phi_node = def phi_node(...)                 │
│    pointer_type = def pointer_type(...)             │
│       REDUCE_OP = def REDUCE_OP(...) Members:       │
│     struct_type = def struct_type(...)              │
│            type = def type(...)                     │
│           undef = def undef(...)                    │
│            user = def user(...)                     │
│           value = def value(...)                    │
╰─────────────────────────────────────────────────────╯

🤔 OK this is getting interesting we have lots of things related to caches and eviction policies. This kinda makes sense. GPUs have a memory hierarchy
* Local
* Shared
* Global

A big part of making GPU computations efficient is about making sure you're cleverly using your cache (or so I'm told)

Ok let's hold off on this part for a while, what's a tensor?

In [49]:
inspect(tl.tensor, methods=True)

╭───────── <class 'triton.language.core.tensor'> ─────────╮
│ def tensor(handle, type: 'dtype'):                      │
│                                                         │
│ to = def to(self, dtype, bitcast=False, _builder=None): │
╰─────────────────────────────────────────────────────────╯

Just a single function `to()`, is this the same as the `torch.Tensor.to()` function commonly used to move data to GPU? Also why did `openai/triton` needs its own `tensor` class? Why not just use `torch`?

In [52]:
inspect(tl.tensor, all=True)

╭────────────────────────── <class 'triton.language.core.tensor'> ──────────────────────────╮
│ def tensor(handle, type: 'dtype'):                                                        │
│                                                                                           │
│          __dict__ = mappingproxy({                                                        │
│                         '__module__': 'triton.language.core',                             │
│                         '_to_dtype': <staticmethod object at 0x7fc5aa1eeb80>,             │
│                         '__init__': <function tensor.__init__ at 0x7fc5aa180ca0>,         │
│                         '__str__': <function tensor.__str__ at 0x7fc5aa180d30>,           │
│                         '__add__': <function tensor.__add__ at 0x7fc5aa180e50>,           │
│                         '__radd__': <function tensor.__radd__ at 0x7fc5aa180ee0>,         │
│                         '__sub__': <function tensor.__sub__ at 0x7fc5aa185040>,           │
│                         '__rsub__': <function tensor.__rsub__ at 0x7fc5aa1850d0>,         │
│                         '__mul__': <function tensor.__mul__ at 0x7fc5aa1851f0>,           │
│                         '__rmul__': <function tensor.__rmul__ at 0x7fc5aa185280>,         │
│                         '__truediv__': <function tensor.__truediv__ at 0x7fc5aa1853a0>,   │
│                         '__rtruediv__': <function tensor.__rtruediv__ at 0x7fc5aa185430>, │
│                         '__floordiv__': <function tensor.__floordiv__ at 0x7fc5aa185550>, │
│                         '__mod__': <function tensor.__mod__ at 0x7fc5aa185670>,           │
│                         '__rmod__': <function tensor.__rmod__ at 0x7fc5aa185790>,         │
│                         '__neg__': <function tensor.__neg__ at 0x7fc5aa1858b0>,           │
│                         '__invert__': <function tensor.__invert__ at 0x7fc5aa1859d0>,     │
│                         '__and__': <function tensor.__and__ at 0x7fc5aa185af0>,           │
│                         '__or__': <function tensor.__or__ at 0x7fc5aa185c10>,             │
│                         '__xor__': <function tensor.__xor__ at 0x7fc5aa185d30>,           │
│                         '__lshift__': <function tensor.__lshift__ at 0x7fc5aa185e50>,     │
│                         '__rshift__': <function tensor.__rshift__ at 0x7fc5aa185f70>,     │
│                         '__gt__': <function tensor.__gt__ at 0x7fc5aa1880d0>,             │
│                         '__rgt__': <function tensor.__rgt__ at 0x7fc5aa1881f0>,           │
│                         '__ge__': <function tensor.__ge__ at 0x7fc5aa188310>,             │
│                         '__rge__': <function tensor.__rge__ at 0x7fc5aa188430>,           │
│                         '__lt__': <function tensor.__lt__ at 0x7fc5aa188550>,             │
│                         '__rlt__': <function tensor.__rlt__ at 0x7fc5aa188670>,           │
│                         '__le__': <function tensor.__le__ at 0x7fc5aa188790>,             │
│                         '__rle__': <function tensor.__rle__ at 0x7fc5aa1888b0>,           │
│                         '__eq__': <function tensor.__eq__ at 0x7fc5aa1889d0>,             │
│                         '__ne__': <function tensor.__ne__ at 0x7fc5aa188af0>,             │
│                         '__getitem__': <function tensor.__getitem__ at 0x7fc5aa188c10>,   │
│                         'to': <function tensor.to at 0x7fc5aa188d30>,                     │
│                         '__dict__': <attribute '__dict__' of 'tensor' objects>,           │
│                         '__weakref__': <attribute '__weakref__' of 'tensor' objects>,     │
│                         '__doc__': None,                                                  │
│                         '__hash__': None                                                  │
│                     })                            

In [74]:
# Back to our kernel

# We multiply a `tl.tensor` with a constexpr block size

block_start = pid * BLOCK_SIZE


NameError: name 'pid' is not defined

In [68]:
BLOCK_SIZE = 1024
block_start = 0
offsets = block_start + tl.arange(0, BLOCK_SIZE, _builder= lambda : ())


AttributeError: 'function' object has no attribute 'get_range'

In [72]:
# How do you pass in a custom builder to debug code locally
tl.arange(5, _builder= range(0,5))

TypeError: arange() missing 1 required positional argument: 'end'

In [66]:
pp.pprint(getsource(tl.load))

('@builtin\n'
 'def load(pointer, mask=None, other=None, cache_modifier="", '
 'eviction_policy="", volatile=False, _builder=None):\n'
 '    """\n'
 '    Return a tensor of data whose values are, elementwise, loaded from '
 'memory at location defined by :code:`pointer`.\n'
 '\n'
 '    :code:`mask` and :code:`other` are implicitly broadcast to '
 ':code:`pointer.shape`.\n'
 '\n'
 '    :code:`other` is implicitly typecast to '
 ':code:`pointer.dtype.element_ty`.\n'
 '\n'
 '    :param pointer: Pointers to the data to be loaded.\n'
 '    :type pointer: Block of dtype=triton.PointerDType\n'
 '    :param mask: if mask[idx] is false, do not load the data at address '
 ':code:`pointer[idx]`.\n'
 '    :type mask: Block of triton.int1, optional\n'
 '    :param other: if mask[idx] is false, return other[idx]\n'
 '    :type other: Block, optional\n'
 '    :param cache_modifier: changes cache option in nvidia ptx\n'
 "    'type cache_modifier: str, optional\n"
 '    """\n'
 '    # mask, other can 

In [67]:
pp.pprint(getsource(tl.store))

('@builtin\n'
 'def store(pointer, value, mask=None, _builder=None):\n'
 '    """\n'
 '    Stores :code:`value` tensor of elements in memory, element-wise, at the '
 'memory locations specified by :code:`pointer`.\n'
 '\n'
 '    :code:`value` is implicitly broadcast to :code:`pointer.shape` and '
 'typecast to :code:`pointer.dtype.element_ty`.\n'
 '\n'
 '    :param pointer: The memory locations where the elements of :code:`value` '
 'are stored.\n'
 '    :type pointer: Block of dtype=triton.PointerDType\n'
 '    :param value: The tensor of elements to be stored.\n'
 '    :type value: Block\n'
 '    :param mask: If mask[idx] is false, do not store :code:`value[idx]` at '
 ':code:`pointer[idx]`.\n'
 '    :type mask: Block of triton.int1, optional\n'
 '    """\n'
 '    # value can be constexpr\n'
 '    value = _to_tensor(value, _builder)\n'
 '    if mask is not None:\n'
 '        mask = _to_tensor(mask, _builder)\n'
 '    return semantic.store(pointer, value, mask, _builder)\n')


In [73]:
@triton.jit
def add_kernel(
    x_ptr,  # *Pointer* to first input vector
    y_ptr,  # *Pointer* to second input vector
    output_ptr,  # *Pointer* to output vector
    n_elements,  # Size of the vector
    BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process
                 # NOTE: `constexpr` so it can be used as a shape value
):
    # There are multiple 'program's processing different data. We identify which program
    # we are here
    pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0
    
    print(type(pid))
    
    # This program will process inputs that are offset from the initial data.
    # for instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # Create a mask to guard memory operations against out-of-bounds accesses
    mask = offsets < n_elements
    # Load x and y from DRAM, masking out any extra elements in case the input is not a
    # multiple of the block size
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    output = x + y
    # Write x + y back to DRAM
    tl.store(output_ptr + offsets, output, mask=mask)